<a href="https://colab.research.google.com/github/littlewhitesea/360PanT/blob/main/360PanT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !git clone https://github.com/littlewhitesea/360PanT.git
%cd pnp-diffusers

/home/zhengbo/code/360PanT/pnp-diffusers


/home/zhengbo/anaconda3/envs/360pant/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
# @title ## 1. Install environment
!pip install -r requirements.txt
!pip install "jax[cuda12_pip]==0.4.23" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

  Using cached importlib_metadata-8.5.0-py3-none-any.whl.metadata (4.8 kB)
  Using cached fsspec-2024.10.0-py3-none-any.whl.metadata (11 kB)
  Using cached zipp-3.21.0-py3-none-any.whl.metadata (3.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.0 MB/s eta 0:00:0000:01m0:16m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 1.8 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 3.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 1.1 MB/s eta 0:00:0000:0100:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 32.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 30.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 26.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 25.6 MB/s eta 0:00:0000:0100:01
   ━━━━

In [9]:
!pip install opencv-python

  Using cached opencv_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
Using cached opencv_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (62.5 MB)


In [6]:
!pip install huggingface_hub==0.25.1

  Using cached huggingface_hub-0.25.1-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-0.25.1-py3-none-any.whl (436 kB)
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.26.5
    Uninstalling huggingface-hub-0.26.5:
      Successfully uninstalled huggingface-hub-0.26.5


In [2]:
# @title ## 2. Generate noisy latent features

# @markdown method choice: 1. PnP; 2. 360PanT
method = "360PanT" # @param {'type':'string'}
path_of_360panorama = "data/original.png" # @param {'type':'string'}
%set_env LD_LIBRARY_PATH="/home/zhengbo/Downloads/cudnn-linux-x86_64-8.9.7.29_cuda11-archive/lib"
command_line = f"python preprocess.py --method {method} --data_path {path_of_360panorama}"
!{command_line}

env: LD_LIBRARY_PATH="/home/zhengbo/Downloads/cudnn-linux-x86_64-8.9.7.29_cuda11-archive/lib"
/home/zhengbo/anaconda3/envs/360pant/lib/python3.11/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
[INFO] loading stable diffusion...


/home/zhengbo/anaconda3/envs/360pant/lib/python3.11/site-packages/diffusers/models/modeling_utils.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(che

In [3]:
# @title ## 3. Generate translated results

import yaml
import os

# @markdown method choice: 1. PnP; 2. 360PanT
method = "360PanT" # @param {'type':'string'}
path_of_360panorama = "data/original.png" # @param {'type':'string'}
prompt = "a photo of castle." # @param {'type':'string'}
path_of_config = "config_pnp.yaml" # @param {'type':'string'}

output_file_name = os.path.splitext(os.path.basename(path_of_360panorama))[0]

output_path = method + '-results/' + output_file_name
with open(path_of_config, 'r') as file:
    yaml_data = yaml.safe_load(file)
    yaml_data['image_path'] = path_of_360panorama
    yaml_data['prompt'] = prompt
    yaml_data['output_path'] = output_path

with open(path_of_config, 'w') as file:
    yaml.dump(yaml_data, file, default_flow_style=False)

command_line = f"python pnp.py --method {method} --config_path {path_of_config}"
!{command_line}

{'device': 'cuda', 'guidance_scale': 7.5, 'image_path': 'data/original.png', 'latents_path': 'latents_forward', 'n_timesteps': 50, 'negative_prompt': 'ugly, blurry, black, low res, unrealistic', 'output_path': '360PanT-results/original', 'pnp_attn_t': 0.5, 'pnp_f_t': 0.8, 'prompt': 'a photo of castle.', 'sd_version': '2.1', 'seed': 1}
Loading SD model
/home/zhengbo/anaconda3/envs/360pant/lib/python3.11/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
SD model loaded
/home/zhengbo/code/360PanT/pnp-diffusers/pnp.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github